In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from lib.opts import opts
from lib.models.model import create_model, load_model, save_model
from lib.models.data_parallel import DataParallel
from lib.logger import Logger
from lib.datasets.dataset_factory import get_dataset
from lib.trains.train_factory import train_factory
from .networks.dlav0 import get_pose_net as get_dlav0
from .networks.pose_dla_dcn import get_pose_net as get_dla_dcn
from .networks.pose_hrnet import get_pose_net as get_pose_net_hrnet
from .networks.resnet_dcn import get_pose_net as get_pose_net_dcn
from .networks.resnet_fpn_dcn import get_pose_net as get_pose_net_fpn_dcn
from .networks.csp_darknet import get_csp_darknet
import _init_paths

In [2]:
import os
import torch
import json
import torch.utils.data
from torchvision.transforms import transforms as T

os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
my_visible_devs = '1'
os.environ['CUDA_VISIBLE_DEVICES'] = my_visible_devs
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
train_factory = {'mot':MotTrainer}

In [ ]:
def save_model(path, epoch, model, optimizer=None):
    if isinstance(model, torch.nn.DataParallel):
        state_dict = model.module.state_dict()
    else:
        state_dict = model.state_dict()
    data = {'epoch':epoch, 'stare_dict':state_dict}
    if optimizer:
        data['optimizer'] = optimizer.state_dict()
    torch.save(data, path)

In [ ]:
def run(opt):
    torch.manual_seed(opt.seed)
    torch.cuda.cudnn.benchmark = not opt.not_cuda_benchmark and not opt.test
    
    f = open(opt.data_cfg)
    data_config = json.load(f)
    
    """
    {
        "root": "tracking/dataset",
        "train":
        {
            "visdrone":"./data/visdrone.train"
        },
        "test_emb":
        {
            "visdrone":"./data/visdrone.test"
        },
        "test":
        {
            "visdrone":"./data/visdrone.test"
        }
}
    """
    trainset_paths = data_config['train']
    dataset_root = data_config['root']
    f.close()
    print("The dataset root is: {}".format(dataset_root))
    
    transforms = T.Compose([T.ToTensor()])
    dataset = MultiScaleJD(opt=opt, root=dataset_root, paths=trainset_paths, img_size = (1088, 608),
                augment = True, transforms = transforms)
    opt = opts().update_dataset_info_and_set_heads(opt, dataset)
    opt.device = device
    opt.gpus = my_visible_devs
    model = create_model(opt.arch, opt.heads, opt.head_conv)
    optimizer = torch.optim.Adam(model.parameters(), opt.lr)
    start_epoch = 0
    if opt.load_model != "":
        model, optimizer, start_epoch = load_model(model, opt.load_model, optimizer, opt.resume, opt.lr, opt.lr_step)
    train_loader = torch.utils.data.DataLoader(dataset = dataset, 
                                               batch_size = opt.batch_size,
                                               shuffle = True,
                                               pin_memory = True,
                                               drop_last = True)
    Trainer = train_factory[opt.task]
    trainer = Trainer(opt=opt, model=model, optimizer=optimizer)
    
    trainer.set_device(opt.gpus, opt.chunk_sizes, device)
    best = 1e10
    for epoch in range(start_epoch+1, opt.num_epochs + 1):
        mark = epoch if opt.save_all else 'last'
        log_dict_train , _ = trainer.train(epoch, train_loader)
        if opt.val_intervals > 0 and epoch% opt.val_intervals == 0:
            save_model(save_path, epoch, model, optimizer)
        else:
            save_model(save_latest_path, epoch, model, optimizer)
        if epoch in opt.lr_step:
            lr = opt.lr * (0.1 ** (opt.lr_step.index(epoch)+1))
            for param_group in optimizer.param_group:
                param_group['lr'] = lr

In [ ]:
class BaseTrainer(object):
    def __init__(self, opt, model, optimizer = None):
        self.opt = opt
        self.model = model
        self.loss_stats, self.loss = self._get_losses(opt)
        self.model_with_loss = ModelWithLoss(model, self.loss)
        
        self.optimizer.add_param_group({'parameter':self.loss.parameters()})
    
    def set_device(self, gpus, chunk_sizes, device):
        dev_ids = [i for i in range(len(gpus))]
        if len(gpus) > 1:
            self.model_with_loss = DataParallel(self.model_with_loss,
                                                device_ids=dev_ids,  
                                                chunk_sizes=chunk_sizes).to(device)
        else:
            self.model_with_loss = self.model_with_loss.to(device)

        for state in self.optimizer.state.values():
            for k, v in state.items():
                if isinstance(v, torch.Tensor):
                    state[k] = v.to(device=device, non_blocking=True)
    
    def run_epoch(self, phase, epoch, data_loader):
        model_with_loss = self.model_with_loss
        if phase == "train":
            model_with_loss.train()
        else:
            if len(self.opt.gpus)>1:
                model_with_loss = self.model_with_loss.module
            model_with_loss.eval()
            torch.cuda.empty_cache()
        
        self.opt = opt
        results = {}
        data_time, batch_time = AverageMeter(), AverageMeter()
        avg_loss_stats = {l: AverageMeter() for l in self.loss_stats}
        num_iters = len(data_loader) if opt.num_iters<0 else opt.num_iters
        end = time.time()
        
        for batch_i, batch in enumerate(data_loader):
            if batch_i >= num_iters:
                break
            
            data_time.update(time.time()-end)
            for k in batch:
                if k!= 'meta':
                    batch[k] = batck[k].to(device = opt.device, non_blocking=True)
            
            output, loss, loss_stats = model_with_loss.forward(batch)
            loss = loss.mean()
            if phase == "train":
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
            
            batch_time.update(time.time()-end)
            end = time.time()
            
            for l in avg_loss_stats:
                avg_loss_stats[l].update(loss_stats[l].mean(), batch['input'].size(0))
            
            scale_idx = data_loader.dataset.batch_i_to_scale_i[batch_i]
            if data_loader.dataset.input_multi_scales is None:
                img_size = Input_WHs[scale_idx]
            else:
                img_size = data_loader.dataset.input_multi_scales[scale_idx]
            print("Image scale in this batch: {}".format(img_size))
            if opt.test:
                self.save_result(output, batch, results)
        
        data_loader.dataset.shuffle()
        ret = {k:v.avg for k, v in avg_loss_stats.items()}
        return ret, results
    
    def train(self, epoch, data_loader):
        return self.run_epoch("train", epoch, data_loader)
    
    def val(self, epoch, data_loadere):
        return self.run_epoch("val", epoch, data_loader)

In [ ]:
class MotTrainer(BaseTrainer):
    def __init__(self, opt, model, optimizer=None):
        super(MotTrainer, self).__init__(opt, model, optimizer = optimizer)
    
    def _get_losses(self, opt):
        if opt.id_weight > 0:
            loss_states = ['loss', 'hm_loss', 'wh_loss', 'off_loss', 'id_loss']
        else:
            loss_states = ['loss', 'hm_loss', 'wh_loss', 'off_loss']
        
        loss = McMotLoss(opt)
        return loss_states, loss
    
    def save_result(self, output, batch, results):
        reg = output['reg'] if self.opt.reg_offset else None
        dets = mot_decode(heatmap = output['hm'], 
                         wh = output['wh'],
                         reg = reg,
                         cat_spec_wh = self.opt.cat_spec_wh,
                         K = self.opt.K)
        dets = dets.detach().cpu().numpy().reshape(1, -1, dets.shape[2])
        dets_out = ctdet_post_process(dets.copy(),
                                     batch['meta']['c'].cpu().numpy(),
                                     batch['meta']['s'].cpu().numpy(),
                                     output['hm'].shape[2],
                                     output['hm'].shape[3],
                                     output['hm'].shape[1])
        results[batch['meta']['img_id'].cpu().numpy()[0]] = dets_out[0]
        

In [ ]:
class McMotLoss(torch.nn.module):
    def __init__(self, opt):
        self.opt = opt
        self.crit = torch.nn.MSELoss() if opt.mse_loss else FocalLoss()
        self.crit_reg = RegL1Loss() if opt.reg_loss == 'l1' else \
        RegLoss() if opt.reg_loss == 'sl1' else None
        
        self.crit_wh = self.crit_reg
        
        if opt.id_weight > 0:
            self.emb_dim = opt.reid_dim
            self.nID_dict = opt.nID_dict
            self.classifier = nn.ModuleDict()
            
            for cls_id, nID in self.nID_dict.items():
                self.classifer[str(cls_id)] = nn.Linear(self.emb_dim, nID)
            
            self.ce_loss = nn.CrossEntropyLoss(ignore_index=-1)
            self.emb_scale_dict = dict()
            for cls_id, nID in self.nID_dict.items():
                self.emb_scale_dict[cls_id] = math.sqrt(2) * math.log(nID-1)
            
            self.s_id = nn.Parameter(-1.05*torch.ones(1))
        
        self.s_det = nn.Paraeter(-1.85*torch.ones(1))
    
    def forward(self, outputs, batch):
        opt = self.opt
        hm_loss, wh_loss, off_loss, reid_loss = 0, 0, 0, 0
        for s in range(opt.num_stack):
            output = outputs[s]
            if not opt.mse_loss:
                output['hm'] = _sigmoid(output['hm'])
            hm_loss += self.crit(output['hm'], batch['hm'])/opt.num_stack
            if opt.wh_weight>0:
                wh_loss += self.crit_reg(output['wh'], batch['reg_mask'],
                                batch['ind'], batch['wh'])/opt.num_stack
            
            if opt.reg_offset and opt.off_weight>0:
                off_loss += self.crit_reg(output['reg'], batch['reg_mask'],
                                batch['ind'], batch['reg'])/opt.num_stack
            
            if opt.id_weight > 0:
                cls_id_map = batch['cls_id_map']
                
                for cls_id , id_num in self.nID_dict.items():
                    inds = torch.where(cls_id_map == cls_id)
                    if inds[0].shape[0] == 0:
                        continue
                    cls_id_head = output['id'][inds[0],:,inds[2],inds[3]]
                    cls_id_head = self.emb_scale_dict[cls_id] * F.normalize(cls_id_head)
                    cls_id_target = batch['cls_tr_ids'][inds[0],:,inds[2],inds[3]]
                    cls_id_pred = self.classifier[str(cls_id)].forward(cls_id_head).contiguous()
                    reid_loss += self.ce_loss(cls_id_pred, cls_id_target) / float(cls_id_target.nelement())
        
        det_loss = opt.hm_weight * hm_loss + opt.wh_weight * wh_loss + \
                    opt.off_weight * off_loss
        
        if opt.id_weight > 0:
            loss = torch.exp(-self.s_det) * det_loss + torch.exp(-self.s_id) * reid_loss\
                    +self.s_det + self.s_id
        else:
            loss = torch.exp(-self.s_det) * det_loss + \
                    +self.s_det 
        
        loss *= 0.5
        
        if opt.id_weight > 0:
            loss_stats = {'loss':loss,
                         'hm_loss':hm_loss,
                         'wh_loss':wh_loss,
                         'off_loss':off_loss,
                         'id_loss':id_loss}
        else:
            loss_stats = {'loss':loss,
                         'hm_loss':hm_loss,
                         'wh_loss':wh_loss,
                         'off_loss':off_loss,
                         }
        return loss, loss_stats
        
                    
            
        

In [ ]:
class ModelWithLoss(torch.nn.Module):
    def __init__(self, model, loss):
        super(ModelWithLoss, self).__init__()
        self.model = model
        self.loss = loss

    def forward(self, batch):
        outputs = self.model.forward(batch['input'])
        loss, loss_stats = self.loss.forward(outputs=outputs, batch=batch)

        return outputs[-1], loss, loss_stats

In [ ]:
def update_dataset_info_and_set_heads(self, opt, dataset):
    input_h, input_w = dataset.default_resolution  
    opt.mean, opt.std = dataset.mean, dataset.std  
    opt.num_classes = dataset.num_classes  

    for reid_id in opt.reid_cls_ids.split(','):
        if int(reid_id) > opt.num_classes - 1:
            print('[Err]: configuration conflict of reid_cls_ids and num_classes!')
            return


    input_h = opt.input_res if opt.input_res > 0 else input_h
    input_w = opt.input_res if opt.input_res > 0 else input_w
    opt.input_h = opt.input_h if opt.input_h > 0 else input_h
    opt.input_w = opt.input_w if opt.input_w > 0 else input_w
    opt.output_h = opt.input_h // opt.down_ratio  
    opt.output_w = opt.input_w // opt.down_ratio
    opt.input_res = max(opt.input_h, opt.input_w)
    opt.output_res = max(opt.output_h, opt.output_w)

    if opt.task == 'mot':
        opt.heads = {'hm': opt.num_classes,
                     'wh': 2 if not opt.cat_spec_wh else 2 * opt.num_classes,
                     'id': opt.reid_dim}
        if opt.reg_offset:
            opt.heads.update({'reg': 2})

        if opt.id_weight > 0:
            opt.nID_dict = dataset.nID_dict

    else:
        assert 0, 'task not defined!'

    return opt

In [ ]:
_model_factory = {
    'dlav0': get_dlav0,  # default DLAup
    'dla': get_dla_dcn,
    'resdcn': get_pose_net_dcn,
    'resfpndcn': get_pose_net_fpn_dcn,
    'hrnet': get_pose_net_hrnet,
    'cspdarknet': get_csp_darknet,
}


def create_model(arch, heads, head_conv):
    num_layers = int(arch[arch.find('_') + 1:]) if '_' in arch else 0 
    arch = arch[:arch.find('_')] if '_' in arch else arch
    get_model = _model_factory[arch]
    model = get_model(num_layers=num_layers, heads=heads, head_conv=head_conv)

    return model


def load_model(model,
               model_path,
               optimizer=None,
               resume=False,
               lr=None,
               lr_step=None):
    """
    """
    start_epoch = 0
    checkpoint = torch.load(model_path, map_location=lambda storage, loc: storage)
    if 'epoch' in checkpoint.keys():
        print('loaded {}, epoch {}'.format(model_path, checkpoint['epoch']))

    if 'state_dict' in checkpoint.keys():
        state_dict_ = checkpoint['state_dict']
    else:
        state_dict_ = checkpoint
    state_dict = {}

    # convert data_parallal to model
    for k in state_dict_:
        if k.startswith('module') and not k.startswith('module_list'):
            state_dict[k[7:]] = state_dict_[k]
        else:
            state_dict[k] = state_dict_[k]
    model_state_dict = model.state_dict()

    # check loaded parameters and created model parameters
    msg = "Only part of weights loaded! please carefully check the weights."
    for k in state_dict:
        if k in model_state_dict:
            if state_dict[k].shape != model_state_dict[k].shape:
                print('Skip loading parameter {}, required shape{}, '
                      'loaded shape{}. {}'.format(
                    k, model_state_dict[k].shape, state_dict[k].shape, msg))
                state_dict[k] = model_state_dict[k]
        else:
            print('Drop parameter {}.'.format(k) + msg)
    for k in model_state_dict:
        if not (k in state_dict):
            print('No param {}.'.format(k) + msg)
            state_dict[k] = model_state_dict[k]
    model.load_state_dict(state_dict, strict=False)

    # resume optimizer parameters
    if optimizer is not None and resume:
        if 'optimizer' in checkpoint:
            optimizer.load_state_dict(checkpoint['optimizer'])
            start_epoch = checkpoint['epoch']
            start_lr = lr
            for step in lr_step:
                if start_epoch >= step:
                    start_lr *= 0.1
            for param_group in optimizer.param_groups:
                param_group['lr'] = start_lr
            print('Resumed optimizer with start lr', start_lr)
        else:
            print('No optimizer parameters in checkpoint.')
    if optimizer is not None:
        return model, optimizer, start_epoch
    else:
        return model

In [ ]:
def save_model(path, epoch, model, optimizer=None):
    if isinstance(model, torch.nn.DataParallel):
        state_dict = model.module.state_dict()
    else:
        state_dict = model.state_dict()

    data = {'epoch': epoch,
            'state_dict': state_dict}

    if not (optimizer is None):
        data['optimizer'] = optimizer.state_dict()

    torch.save(data, path)